[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/feature/milvus/Demo/Milvus.ipynb)

In [1]:
!pip install indox
!pip install openai
!pip install pymilvus

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indox
```
2. **Activate the virtual environment:**
```bash
indox_judge\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
```

2. **Activate the virtual environment:**
    ```bash
   source indox/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
INDOX_API_KEY = os.getenv("INDOX_API_KEY")

## Initial Setup

The following imports are essential for setting up the Indox application. These imports include the main Indox retrieval augmentation module, question-answering models, embeddings, and data loader splitter.


In [3]:
from indox import IndoxRetrievalAugmentation
from indox.llms import IndoxApi
from indox.embeddings import IndoxApiEmbedding
from indox.data_loader_splitter import ClusteredSplit

In [4]:
indox = IndoxRetrievalAugmentation()
openai_qa_indox = IndoxApi(api_key=INDOX_API_KEY)
embed_openai_indox = IndoxApiEmbedding(api_key=INDOX_API_KEY, model="text-embedding-3-small")


INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            
INFO: Initialized IndoxOpenAIEmbedding with model: text-embedding-3-small


In [5]:
#!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

In [6]:
file_path = "sample.txt"
loader_splitter = ClusteredSplit(file_path=file_path, embeddings=embed_openai_indox, summary_model=openai_qa_indox)
docs = loader_splitter.load_and_chunk()

INFO: ClusteredSplit initialized successfully


In [7]:
from indox.vector_stores import Milvus

## Vector Store Connection and Document Storage

In this step, we connect the Indox application to the vector store and store the processed documents.


In [8]:
db = Milvus(collection_name="sample", embedding_model=embed_openai_indox)
db.add(docs=docs)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: c498e77ae0e84f7aae009a723b8c556f


INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            
INFO: Connection to the vector store database established successfully


In [9]:
Indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            
INFO: Connection to the vector store database established successfully


## Querying and Interpreting the Response

In this step, we query the Indox application with a specific question and use the QA model to get the response. 



In [12]:
query = "How cinderella reach her happy ending?"
retriever = indox.RetrievalAugmentation(vector_database=db, llm=openai_qa_indox, top_k=5)
retriever.invoke(query)

INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings texts using engine: text-embedding-3-small
INFO: Generating answer without document relevancy filter
INFO: Query answered successfully


"In the story of Cinderella, she reaches her happy ending through her kindness, resilience, and unwavering belief in goodness. Despite facing hardships and mistreatment from her stepfamily, Cinderella remains true to herself and never loses hope. With the help of her fairy godmother, she is able to attend the royal ball and capture the heart of the prince. Ultimately, Cinderella's pure heart and inner strength lead her to her happy ending, where she is rewarded for her virtues and finds love and happiness."